# การประเมินผล OCR (Optical Character Recognition)

โน้ตบุ๊กนี้แสดงวิธีการประเมินประสิทธิภาพของระบบ OCR โดยใช้ตัวชี้วัดสองตัว:

- CER (Character Error Rate) หรืออัตราความผิดพลาดระดับตัวอักษร
- ความแม่นยำ (Accuracy)

## ติดตั้งไลบรารีที่จำเป็น

In [ ]:
%%capture
!pip install torch
!pip install torchmetrics

นำเข้าไลบรารีและการกำหนดฟังก์ชัน

- สร้างคลาส `CharErrorRate` และ คำนวณความผิดพลาด
- หรือใช้ฟังก์ชั่น `char_error_rate` เพื่อคำนวณ CER โดยตรง
- สำหรับ accuracy สามารถใช้ `==` เพื่อเทียบ string 2 ตัวได้เลย

In [ ]:
import numpy as np
import torch
from torchmetrics.text import CharErrorRate

def calculate_cer(preds: list, targets: list):
    cer = CharErrorRate() # Initialize the CharErrorRate metric
    cer_val = cer(preds, targets) # Calculate CER
    return cer_val.tolist()

In [ ]:
preds = ["สวัสดีครับ ยินดีต้อนรับ"]
targets = ["สวัสดีคับ ยินดีต้อนรบ"]

In [ ]:
cer = calculate_cer(preds, targets)
print(f"Character Error Rate (CER): {cer:.2%}")

In [ ]:
# Or use the build-in function to calculate CER
from torchmetrics.functional.text import char_error_rate

cer_rate = char_error_rate(preds, targets)
print(f"\nDetailed CER calculation: {cer_rate:.2%}")

In [ ]:
accuracy = [int(pred == target) for pred, target
            in zip(preds, targets)]  # 1 if the strings are equal, 0 otherwise
print(f"\nAccuracy: {np.mean(accuracy)}")

## Deskew เพื่อปรับภาพให้ตรง

In [ ]:
!pip install deskew

In [ ]:
import os
import os.path as op
from glob import glob
from pathlib import Path
from tqdm.auto import tqdm

import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.transform import rotate
from deskew import determine_skew

In [ ]:
def deskew_image(input_path, output_path):
    """
    ฟังก์ชั่นสำหรับอ่านและหมุนภาพให้ตรงเพื่อใช้สำหรับ OCR
    """
    image = io.imread(input_path) # อ่านภาพจากไฟล์
    grayscale = rgb2gray(image) # แปลงเป็นภาพขาวดำ
    angle = determine_skew(grayscale) # หามุมเอียงของภาพ
    rotated = rotate(image, angle, resize=True) * 255 # หมุนภาพเพื่อแก้ไขความเอียง
    # บันทึกภาพที่ผ่านการประมวลผล
    io.imsave(output_path, rotated.astype(np.uint8))

def process_images(input_dir, output_dir):
    """
    ฟังก์ชันสำหรับประมวลผลภาพทั้งหมดในโฟลเดอร์
    """
    os.makedirs(output_dir, exist_ok=True)
    input_paths = glob(os.path.join(input_dir, "*.png")) + \
                  glob(os.path.join(input_dir, "*.jpg")) + \
                  glob(os.path.join(input_dir, "*.jpeg"))
    for input_path in tqdm(input_paths):
        output_path = op.join(output_dir, f"{Path(input_path).stem}.png")
        deskew_image(input_path, output_path)

In [ ]:
process_images("/path/to/input_folder", "output")